In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import dask
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

import papermill as pm

import validate_case

In [3]:
cases = [
    "smyle.control.perlmutter.004",
    "smyle.oae.perlmutter.alk-forcing-lat-range-basin_000.001",
]

In [4]:
USER = os.environ["USER"]
SCRATCH = f"/pscratch/sd/{USER[0]}/{USER}"
memory = "32GB"
walltime = "02:00:00"
account = "m4746"

# SLURMCluster doesn't seem to generate at usable jobscript
# hack it
job_directives_skip = ["-J", "-e", "-o", "-p","-A", "-n", "--cpus-per-task", "--mem", "-t"]   
job_extra_directives = [
    "-J dask-worker",
    f"-A {account}",
    f"-e {SCRATCH}/dask-workers/dask-worker-%J.err",
    f"-o {SCRATCH}/dask-workers/dask-worker-%J.out",
    f"--time={walltime}",
    f"--mem={memory}",    
    "--qos=shared",    
    "--nodes=1",
    "--ntasks=1",
    "--ntasks-per-node=64", # shared queue only allows half node — don't know if this is really needed
    "--cpus-per-task=1",
    "--constraint=cpu",
]

cluster = SLURMCluster(
    cores=1,
    processes=1,    
    memory=memory,
    queue="debug",
    account=account,
    walltime=walltime,
    job_directives_skip=job_directives_skip,
    job_extra_directives=job_extra_directives,
)

cluster.scale(12)
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://128.55.64.38:8787/status,
Dashboard: http://128.55.64.38:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.55.64.38:44099,Workers: 0
Dashboard: http://128.55.64.38:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
clobber = False
for case in cases:
    is_oae_run = ".oae." in case
    zarr_store = validate_case.zarr_validation_data(case)
    if not os.path.exists(zarr_store) or clobber:
        validate_case.validator(case, is_oae_run)        

working on DIC_ALT_CO2...
working on ALK_ALT_CO2...


In [8]:
clobber = True

dir_out = "case-info/validation"
os.makedirs(dir_out, exist_ok=True)

for case in cases:   
    nb_out = f"{dir_out}/validation_{case}.ipynb"
    if os.path.exists(nb_out) and not clobber:
        continue
    
    zarr_store = validate_case.zarr_validation_data(case)    
    if os.path.exists(zarr_store):
        pm.execute_notebook(
           "plot_case_validation.ipynb",
           nb_out,
           parameters=dict(case=case)
        )

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]